# Wide Format Conversion


In [1]:
import os
import pickle
import re
from pathlib import Path
import pandas as pd
from tsfresh import extract_features

In [2]:
CURRENT_DIR = Path.cwd()
PARENT_DIR = CURRENT_DIR.parent
STRESS_DATA_DIR = PARENT_DIR / "P02_data" / "T01_explore" / "S04"
print(STRESS_DATA_DIR)

c:\Users\admin\Coding\research\weld-ml\src\P02_data\T01_explore\S04


In [3]:
# Load stress data
stress_filepath = STRESS_DATA_DIR / "S04_residual_stress_imputed_20260104_114536.xlsx"
_data = pd.read_excel(stress_filepath)
_data

,idx_excel_post,section,sample_no,location,R,W,D,sigma_x_post,FWHM_post,idx_excel_pre,sigma_x_pre,FWHM_pre,diff_sigma_x
0,4,AA5052,1,1,1400,60,10,13.0,2.55,2,-15.0,2.50,28.0
1,25,AA5052,2,1,1400,60,15,16.0,2.51,9,2.0,2.47,14.0
2,46,AA5052,3,1,1400,60,20,19.0,2.47,16,9.0,2.48,10.0
3,67,AA5052,4,1,1400,70,10,20.0,2.45,23,10.0,2.48,10.0
4,88,AA5052,5,1,1400,70,15,6.0,2.47,30,0.0,2.49,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1129,1050,Center,50,7,1600,70,15,2.0,2.45,0,0.0,0.00,2.0
1130,1071,Center,51,7,1600,70,20,2.0,2.45,0,0.0,0.00,2.0
1131,1092,Center,52,7,1600,80,10,5.0,2.54,0,0.0,0.00,5.0
1132,1113,Center,53,7,1600,80,15,1.0,2.41,0,0.0,0.00,1.0


In [4]:
# Pivot the data to have multi-level columns for section and location
data1 = _data.pivot_table(
    index=["sample_no", "R", "W", "D"],
    columns=["section", "location"],
    values="diff_sigma_x",
).reset_index()
data1

section  sample_no     R   W   D AA5052                              \
location                              1          2          3     4   
0                1  1400  60  10   28.0  12.000000  33.000000  42.0   
1                2  1400  60  15   14.0  10.000000  26.000000  27.0   
2                3  1400  60  20   10.0  14.000000  14.000000  25.0   
3                4  1400  70  10   10.0  36.000000  22.000000  11.0   
4                5  1400  70  15    6.0  14.000000   8.000000   7.0   
5                6  1400  70  20   15.0  11.000000 -13.000000  -1.0   
6                7  1400  80  10   63.0  45.000000  24.000000  31.0   
7                8  1400  80  15   14.0  11.000000  13.000000  31.0   
8                9  1400  80  20   26.0  40.000000  32.000000  29.0   
9               10  1500  60  10   23.0  30.000000  22.000000  20.0   
10              11  1500  60  15   23.0  34.000000  23.000000  43.0   
11              12  1500  60  20   23.0  27.000000  45.000000  39.0   
12              13  1500  70  10   23.0  26.000000  28.000000  39.0   
13              14  1500  70  15   25.0  25.320755  45.000000  31.0   
14              15  1500  70  20   25.0  37.000000  56.000000  47.0   
15              16  1500  80  10   40.0  48.000000  56.000000  44.0   
16              17  1500  80  15   22.0  40.000000  46.000000  46.0   
17              18  1500  80  20   31.0  21.000000  27.000000  35.0   
18              19  1600  60  10   41.0  37.000000  56.000000  36.0   
19              20  1600  60  15   19.0  35.000000  31.000000  33.0   
20              21  1600  60  20   44.0  35.000000  19.000000  44.0   
21              22  1600  70  10   30.0  27.000000  37.000000  61.0   
22              23  1600  70  15   11.0  25.000000  44.000000  60.0   
23              24  1600  70  20    9.0  21.000000  54.000000  51.0   
24              25  1600  80  10   47.0  36.000000  43.000000  62.0   
25              26  1600  80  15   21.0  37.000000  33.000000  32.0   
26              27  1600  80  20   13.0  21.000000  18.000000  32.0   
27              28  1400  60  10   31.0   5.000000  29.000000  21.0   
28              29  1400  60  15   42.0  14.000000   4.000000  15.0   
29              30  1400  60  20   45.0  -6.000000  42.000000  61.0   
30              31  1400  70  10   37.0  22.000000  38.000000  49.0   
31              32  1400  70  15   19.0  22.000000  14.000000  26.0   
32              33  1400  70  20   26.0  18.000000  17.000000  26.0   
33              34  1400  80  10   45.0  21.000000   8.000000  15.0   
34              35  1400  80  15   26.0  27.000000  38.000000  11.0   
35              36  1400  80  20   31.0  38.000000  20.000000  37.0   
36              37  1500  60  10   59.0 -13.000000  12.000000  22.0   
37              38  1500  60  15   23.0  24.000000  25.000000  37.0   
38              39  1500  60  20   41.0  41.000000  36.000000  23.0   
39              40  1500  70  10   10.0  10.000000 -11.000000 -14.0   
40              41  1500  70  15   17.0  24.000000  20.000000  23.0   
41              42  1500  70  20   25.0  46.000000  42.000000  35.0   
42              43  1500  80  10   25.0  29.000000  34.000000  52.0   
43              44  1500  80  15    5.0   2.000000  -1.000000  12.0   
44              45  1500  80  20   20.0   9.000000   1.000000  15.0   
45              46  1600  60  10   29.0  51.000000  14.698113  37.0   
46              47  1600  60  15   22.0  29.000000   7.000000  18.0   
47              48  1600  60  20   64.0  36.000000  25.000000  32.0   
48              49  1600  70  10   22.0  48.000000  26.000000  28.0   
49              50  1600  70  15   16.0   1.000000   4.000000   2.0   
50              51  1600  70  20    1.0 -10.000000   8.000000  17.0   
51              52  1600  80  10   54.0  14.000000  27.000000  -1.0   
52              53  1600  80  15   19.0  47.000000  35.000000  30.0   
53              54  1600  80  20   43.0  63.000000   9.000000   6.0   

section                

In [5]:
cols = [col[0] + "_L" + str(col[1]) if col[0] not in ["sample_no", "R", "W", "D"] else col[0] for col in data1.columns.to_flat_index()]
print(cols)

['sample_no', 'R', 'W', 'D', 'AA5052_L1', 'AA5052_L2', 'AA5052_L3', 'AA5052_L4', 'AA5052_L5', 'AA5052_L6', 'AA5052_L7', 'AA6061_L1', 'AA6061_L2', 'AA6061_L3', 'AA6061_L4', 'AA6061_L5', 'AA6061_L6', 'AA6061_L7', 'Center_L1', 'Center_L2', 'Center_L3', 'Center_L4', 'Center_L5', 'Center_L6', 'Center_L7']


In [6]:
data1.columns = cols
data1

,sample_no,R,W,D,AA5052_L1,AA5052_L2,AA5052_L3,AA5052_L4,AA5052_L5,AA5052_L6,...,AA6061_L5,AA6061_L6,AA6061_L7,Center_L1,Center_L2,Center_L3,Center_L4,Center_L5,Center_L6,Center_L7
0,1,1400,60,10,28.0,12.000000,33.000000,42.0,23.000000,10.0,...,-24.000000,12.000000,-2.942308,12.0,26.000000,29.0,24.0,19.000000,6.0,6.0
1,2,1400,60,15,14.0,10.000000,26.000000,27.0,21.000000,-4.0,...,2.000000,30.000000,-83.000000,17.0,25.000000,23.0,14.0,10.000000,3.0,2.0
2,3,1400,60,20,10.0,14.000000,14.000000,25.0,11.000000,3.0,...,-35.000000,-18.000000,29.000000,12.0,28.000000,21.0,22.0,12.000000,16.0,8.0
3,4,1400,70,10,10.0,36.000000,22.000000,11.0,14.000000,23.0,...,100.000000,-35.000000,-46.000000,20.0,24.000000,29.0,18.0,11.000000,18.0,3.0
4,5,1400,70,15,6.0,14.000000,8.000000,7.0,13.000000,12.0,...,64.000000,-33.000000,31.000000,14.0,4.000000,8.0,11.0,9.000000,4.0,6.0
5,6,1400,70,20,15.0,11.000000,-13.000000,-1.0,10.000000,-20.0,...,46.000000,-63.000000,-76.000000,11.0,12.000000,13.0,4.0,7.000000,2.0,12.0
6,7,1400,80,10,63.0,45.000000,24.000000,31.0,19.000000,46.0,...,-12.169811,-87.000000,-90.000000,12.0,15.000000,17.0,20.0,12.000000,6.0,-1.0
7,8,1400,80,15,14.0,11.000000,13.000000,31.0,5.000000,8.0,...,-28.000000,16.000000,-30.000000,18.0,15.000000,14.0,7.0,14.000000,16.0,5.0
8,9,1400,80,20,26.0,40.000000,32.000000,29.0,32.000000,13.0,...,15.000000,7.000000,33.000000,17.0,21.000000,18.0,24.0,13.000000,4.0,-5.0
9,10,1500,60,10,23.0,30.000000,22.000000,20.0,14.000000,18.0,...,-44.000000,-46.000000,19.000000,11.0,18.000000,9.0,10.0,6.000000,-2.0,1.0


In [7]:
# Pivot the data to have mean values for each section
data2 = _data.pivot_table(
    index=["sample_no"],
    columns=["section"],
    values="diff_sigma_x",
    aggfunc="mean",
).reset_index()
data2

data2.columns = [col + "_mean" if col != "sample_no" else col for col in data2.columns.to_flat_index()]
data2

,sample_no,AA5052_mean,AA6061_mean,Center_mean
0,1,23.714286,16.293956,17.428571
1,2,12.000000,-28.428571,13.428571
2,3,11.857143,-8.428571,17.000000
3,4,17.714286,8.857143,17.571429
4,5,10.000000,26.000000,8.000000
5,6,0.285714,-38.142857,8.714286
6,7,32.857143,-37.309973,11.571429
7,8,13.142857,-21.142857,12.714286
8,9,25.714286,17.142857,13.142857
9,10,20.571429,-35.571429,7.571429


In [8]:
data = data1.merge(data2, on="sample_no", how="left")
data

,sample_no,R,W,D,AA5052_L1,AA5052_L2,AA5052_L3,AA5052_L4,AA5052_L5,AA5052_L6,...,Center_L1,Center_L2,Center_L3,Center_L4,Center_L5,Center_L6,Center_L7,AA5052_mean,AA6061_mean,Center_mean
0,1,1400,60,10,28.0,12.000000,33.000000,42.0,23.000000,10.0,...,12.0,26.000000,29.0,24.0,19.000000,6.0,6.0,23.714286,16.293956,17.428571
1,2,1400,60,15,14.0,10.000000,26.000000,27.0,21.000000,-4.0,...,17.0,25.000000,23.0,14.0,10.000000,3.0,2.0,12.000000,-28.428571,13.428571
2,3,1400,60,20,10.0,14.000000,14.000000,25.0,11.000000,3.0,...,12.0,28.000000,21.0,22.0,12.000000,16.0,8.0,11.857143,-8.428571,17.000000
3,4,1400,70,10,10.0,36.000000,22.000000,11.0,14.000000,23.0,...,20.0,24.000000,29.0,18.0,11.000000,18.0,3.0,17.714286,8.857143,17.571429
4,5,1400,70,15,6.0,14.000000,8.000000,7.0,13.000000,12.0,...,14.0,4.000000,8.0,11.0,9.000000,4.0,6.0,10.000000,26.000000,8.000000
5,6,1400,70,20,15.0,11.000000,-13.000000,-1.0,10.000000,-20.0,...,11.0,12.000000,13.0,4.0,7.000000,2.0,12.0,0.285714,-38.142857,8.714286
6,7,1400,80,10,63.0,45.000000,24.000000,31.0,19.000000,46.0,...,12.0,15.000000,17.0,20.0,12.000000,6.0,-1.0,32.857143,-37.309973,11.571429
7,8,1400,80,15,14.0,11.000000,13.000000,31.0,5.000000,8.0,...,18.0,15.000000,14.0,7.0,14.000000,16.0,5.0,13.142857,-21.142857,12.714286
8,9,1400,80,20,26.0,40.000000,32.000000,29.0,32.000000,13.0,...,17.0,21.000000,18.0,24.0,13.000000,4.0,-5.0,25.714286,17.142857,13.142857
9,10,1500,60,10,23.0,30.000000,22.000000,20.0,14.000000,18.0,...,11.0,18.000000,9.0,10.0,6.000000,-2.0,1.0,20.571429,-35.571429,7.571429


In [9]:
data.to_excel("S01_residual_stress_wide_format.xlsx", index=False)